This file reads a processed csv file and starts the animation/visualisation of the sensorfloor data for each sensor+direction:

* acceleration x,y,z
* gyroscope x,y,z
* magnetometer x,y,z

All animations can be executed at the same time (but this slows down the animation)

TO DO:

* plot vicon position within the animation
* add technical MIN and MAX values to colorbar

!SAVING TAKES WAY TOO LONG (since the animation is too long)

If you want to save the animation as a video file, to a screen capture

In [ ]:

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

In [ ]:
# Read Data
s = 'Data_aswin_processed.csv'
Data = pd.read_csv(s)

In [ ]:
#Just to be sure, that the data is sorted
Data = Data.sort_values(by = 'timestamp')
Data = Data.reset_index()

In [ ]:
# Class for animation objects

# plot in a new window, not within the notebook
%matplotlib qt 
class AnimatedScatter(object):
    """An animated scatter plot for visualisation of the sensorfloor data."""
    def __init__(self, Data, sensor):
        #Dataframe of sensordata
        self.data = Data
        #df contains the last measurement, that was added to the plot
        self.df = self.setup_sensorfloor()
        #which sensor to be plotted (ax,ay,az,gx,gy,gz,mx,my,mz)
        self.sensor = sensor
        #get min and max value for the colorbar
        self.min = self.data[self.sensor].min()
        self.max = self.data[self.sensor].max()
        # Setup the figure and axes...
        self.fig, [self.ax, self.cax] = plt.subplots(1,2,figsize=(4, 4),gridspec_kw={"width_ratios":[50,1]})
        cmap = matplotlib.cm.jet
        norm = matplotlib.colors.Normalize(vmin=self.min, vmax=self.max)

        cb1 = matplotlib.colorbar.ColorbarBase(self.cax, cmap=cmap,
                                norm=norm,
                                orientation='vertical')
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=0.1, frames = len(self.data), repeat = False, 
                                          init_func=self.setup_plot, blit=True)
    def toIndex(self,node_id, strip_id):
        '''Takes node_id and strip_id and returns the corresponding index of this node'''
        i = (node_id - 2) + (strip_id-2)*14
        return i
    
    def setup_sensorfloor(self):
        '''Initialize Dataframe for the last measurment to be added to the animation.'''
        positions = [[i,j] for i in range(2,16) for j in range(2,24)]
        positions = np.reshape(positions, (len(positions),2))
        sensorfloor = pd.DataFrame(columns =['node_id', 'strip_id', 'color'])
        sensorfloor['node_id'] = positions[:,0]
        sensorfloor['strip_id'] = positions[:,1]
        sensorfloor['color'] = np.zeros(len(sensorfloor))
        return sensorfloor
        
    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        x = self.df['node_id']
        y = self.df['strip_id']
        c = self.df['color']
        self.scat = self.ax.scatter(x, y, c=c, s= 200,vmin = self.min, vmax = self.max, marker = 's',
                                    cmap="jet")
        self.ax.set_xlabel('node_id')
        self.ax.set_ylabel('strip_id')
        self.ax.set(xlim=(1, 16), ylim=(1, 25))
        self.cax.set_title(self.sensor)
        self.title = self.ax.text(0.5,0.95, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                 transform=self.ax.transAxes, ha="center")
        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        #self.ax.clear() #setup_plot will be called 2 times!
        return self.scat, self.title,


    def update(self, i):
        """Update the scatter plot."""
        #Get next datapoint
        tmp = self.data.iloc[i]
        #update data
        self.df.loc[self.toIndex(tmp['node_id'], tmp['strip_id']), 'color'] = tmp[self.sensor]
        # Set color
        self.scat.set_array(self.df['color'])
        self.title.set_text(' Timestamp: '+tmp.loc['timestamp'])
        #return series of updated scatterplots and titles
        return self.scat, self.title,
    
    def saveAnimatedScatter(self, fps = 15):
        '''Saves the animation to a mp4 file.'''
        Writer = animation.FFMpegWriter(fps)
        self.ani.save('test.mp4', writer = Writer)




In [ ]:
mx = AnimatedScatter(Data,'mx')
#mz = AnimatedScatter(Data,'mz')
#my = AnimatedScatter(Data,'my')
#ax = AnimatedScatter(Data,'ax')
#az = AnimatedScatter(Data,'az')
#ay = AnimatedScatter(Data,'ay')
#gx = AnimatedScatter(Data,'gx')
#gz = AnimatedScatter(Data,'gz')
#gy = AnimatedScatter(Data,'gy')
plt.show()
#a.saveAnimatedScatter()